# Ejercicios Extraclase Avance 2
------------------------------------------------------------
### Estudiantes:

- #### Ana Melissa Vásquez Rojas 
- #### Daniel Duarte Cordero 

## Pregunta 1

Determina si una matriz cuadrada $A ∈ ℝ^{m×m}$  tiene una única factorización LU.

Basado en el teorema: Una matriz cuadrada A tiene una única factorización LU si y solo si es invertible, si y solo si
$rango(A) = m$

### Parámetros:
- `A`: Matriz cuadrada $ m \times m $.

### Retorna:

- `tf=1` si \( A \) tiene una única factorización LU.  
- `tf=0` si no la tiene.



In [5]:


import numpy as np

def tieneUnicaFactLU(A):

    A = np.array(A)
    m, n = A.shape

    # Verificar que A sea cuadrada
    if m != n:
        return 0

    # Verificación del rango de la matriz principal
    if np.linalg.matrix_rank(A) != m:
        return 0  # La matriz tiene rango diferente de m, entonces no hay LU única
    else:
        return 1  # Sí tiene una única factorización LU
    
# Caso 1: matriz con LU única
A1 = np.array([
    [2, 1, 1, 0, 0],
    [4, 3, 3, 1, 0],
    [8, 7, 9, 5, 1],
    [6, 7, 9, 8, 4],
    [2, 2, 3, 4, 7]
])

# Caso 2: matriz sin LU única (submatrices principales no invertibles)
A2 = np.array([
    [1, 2, 3, 4, 5],
    [2, 4, 6, 8, 10],
    [1, 1, 1, 1, 1],
    [3, 6, 9, 12, 15],
    [0, 0, 0, 0, 0]
])

print("A1 tiene LU única:", "tf=", tieneUnicaFactLU(A1))  # 1
print("A2 tiene LU única:", "tf=",tieneUnicaFactLU(A2))  # 0



A1 tiene LU única: tf= 1
A2 tiene LU única: tf= 0


## Pregunta 2

Implemente el método de Thomas para resolver el sistema de ecuaciones $Ax = b$, donde $A$ es una matriz tridiagonal.

### Fundamento teórico

El método de Thomas es una versión especializada del método de eliminación de Gauss que se aplica exclusivamente a sistemas lineales cuya matriz de coeficientes es **tridiagonal**, es decir, solo tiene elementos distintos de cero en la diagonal principal y en las dos diagonales adyacentes.

Este método divide el procedimiento en dos etapas:

1. **Eliminación hacia adelante**: se transforma el sistema en uno equivalente con matriz triangular superior, simplificando las ecuaciones en forma recursiva.
2. **Sustitución hacia atrás**: se utiliza para resolver el sistema resultante y encontrar el vector solución.

El algoritmo aprovecha la estructura tridiagonal para reducir tanto la complejidad computacional como el uso de memoria, logrando una eficiencia de orden $\mathcal{O}(n)$.

### Prueba del funcionamiento

Para comprobar el correcto funcionamiento del método de Thomas, se resolverá el sistema \(Ax = d\) donde:

- La matriz $\mathbf{A}$ es tridiagonal de tamaño $100 \times 100$, con $5$ en la diagonal principal y $1$ en las diagonales superior e inferior.
- El vector $\mathbf{d}$ es de tamaño $100$, con los valores $-12$ en las primeras y últimas posiciones, y $-14$ en las posiciones intermedias.



Este sistema se corresponde con el que aparece en la imagen proporcionada, y se usará como caso de prueba para verificar que la implementación del método es correcta.

### Parámetros

- `A`: Matriz tridiagonal $ n \times n $ que representa el sistema.
- `b`: Vector columna de tamaño $ n $, correspondiente al lado derecho del sistema.

### Retorna

- `x`: Vector solución $ x $ del sistema $Ax = b$, de tamaño $ n $.



In [7]:
import numpy as np

def extract_diagonal(matrix, diag_index):
    #Extrae una diagonal específica de una matriz.
    return np.diag(matrix, k=diag_index)


def thomas(A, b):
    """Resuelve un sistema tridiagonal de ecuaciones lineales usando el algoritmo de Thomas.
    
    Args:
        A (np.array): Matriz tridiagonal del sistema (n x n).
        b (np.array): Vector del lado derecho (n).
    
    Returns:
        np.array: Solución del sistema (vector x).
    """
    n = len(b)  # Tamaño del sistema (número de ecuaciones)

    # Extracción de las diagonales:
    a = extract_diagonal(A, -1)  # Subdiagonal (elementos bajo la diagonal principal)
    c = extract_diagonal(A, 1)    # Superdiagonal (elementos sobre la diagonal principal)
    d = b.copy()                 # Copia del vector del lado derecho (para no modificar el original)
    b = extract_diagonal(A, 0)    # Diagonal principal (sobrescribe b, ahora es la diagonal)

    # Inicialización de vectores para el algoritmo:
    p = np.zeros(n-1)  # Coeficientes modificados (forward sweep)
    q = np.zeros(n)    # Términos independientes modificados (forward sweep)

    # Paso 1: Forward sweep (cálculo de p y q)
    # Primer elemento (caso especial i=0)
    p[0] = c[0] / b[0]
    q[0] = d[0] / b[0]

    # Elementos intermedios (i=1 hasta i=n-2)
    for i in range(1, n-1):
        denom = b[i] - a[i-1] * p[i-1]  # Denominador común
        p[i] = c[i] / denom
        q[i] = (d[i] - a[i-1] * q[i-1]) / denom

    # Último elemento (i=n-1)
    denom = b[n-1] - a[n-2] * p[n-2]
    q[n-1] = (d[n-1] - a[n-2] * q[n-2]) / denom

    # Paso 2: Backward substitution (solución hacia atrás)
    x = np.zeros(n)
    x[n-1] = q[n-1]  # Último valor de x (xn = qn)

    # Iteramos desde el penúltimo elemento hasta el primero (i=n-2 hasta i=0)
    for i in range(n-2, -1, -1):
        x[i] = q[i] - p[i] * x[i+1]
    return x.reshape(-1, 1)
# Función para generar la matriz tridiagonal y el vector b solicitada
def matrix_generator():
    A = np.zeros((100,100))
    A[0,0] = 5
    A[0,1] = 1

    for k in range(1,99):
        A[k,k]= 5
        A[k,k-1] = 1
        A[k,k+1] = 1
    
    A[99,99] = 5 
    A[99,98] = 1
    b = np.zeros((100,1))
    b[0] = -12
    b[-1] = -12
    
    for j in range(1,99):
        b[j] = -14
    return A, b

# Resolver el sistema
A, b = matrix_generator()
x = thomas(A, b)
print("Solucion:", x) 

Solucion: [[-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]]


## Pregunta 3

Implementa la factorización $QR$ de una matriz cuadrada $ A \in \mathbb{R}^{m \times m} $ utilizando el proceso de ortogonalización de Gram-Schmidt.

### Fundamento teórico

La factorización $ QR $ permite expresar cualquier matriz cuadrada $ A $ como el producto:

$A = QR$

- $ Q \in \mathbb{R}^{m \times m} $ es una matriz ortogonal, es decir, $ Q^T Q = Q Q^T= Im $,
- $ R \in \mathbb{R}^{m \times m} $ es una matriz triangular superior.

Esta descomposición se obtiene mediante el proceso de Gram-Schmidt:

$
\begin{align*}
u_1 &= a_1, \\
e_1 &= \frac{u_1}{\|u_1\|_2}, \\
u_k &= a_k - \sum_{j=1}^{k-1} \langle a_k, e_j \rangle e_j, \\
e_k &= \frac{u_k}{\|u_k\|_2}, \quad \text{para } k = 2, 3, ..., m.
\end{align*}
$

### Parámetros

- `A`: Matriz cuadrada $ m \times m $.

### Retorna

- `Q`: Matriz ortogonal $ Q $.
- `R`: Matriz triangular superior $ R $.


In [61]:
import numpy as np

def fact_qr(A):
    A = np.array(A, dtype=float)
    m, n = A.shape
    Q = np.zeros((m, n))  
    R = np.zeros((n, n))

    for k in range(n):
        ak = A[:, k]       # Paso 1: tomar columna ak
        uk = ak.copy()     # Inicializar uk como ak

        aux = np.zeros(m)
        for j in range(k):
            ej = Q[:, j]                        # Paso 2: ej 
            R[j, k] = ak @ ej                 # producto punto <ak, ek>
            aux += R[j, k] * ej                  # Sumatoria
        uk = ak - aux
        
        R[k, k] = np.linalg.norm(uk)            # Paso 3: ||uk||
        if R[k, k] == 0:
            raise ValueError("Columna linealmente dependiente detectada")

        ek = uk / R[k, k]                       # Paso 4: ek = uk / ||uk||
        Q[:, k] = ek

    return Q, R



    
A = A = [
    [  2, -1,  3,  0,  1, -2,  4, -3,  5,   6],
    [ -3,  5, -2,  1,  4, -6,  7, -8,  9,  10],
    [  1, -2,  4, -3,  5, -7,  8, -9, 10,  11],
    [  4, -3,  1,  2, -5,  6, -7,  8, -9,  10],
    [ -5,  7, -4,  6,  3, -1,  2, -8,  9, -10],
    [  6, -4,  5, -7,  8,  1, -3,  2, -9,  10],
    [ -7,  9, -6,  8, -10, 11,  3, -5,  4, -12],
    [  8, -6,  7, -9, 10, -12, 13,  5, -4,   3],
    [ -9, 11, -8, 10, -12, 14, -16, 17,  6,  -5],
    [ 10, -8,  9, -11, 12, -14, 15, -17, 18,  7]
]

Q, R = fact_qr(A)
print('Q=', Q)
print('R=', R)



Q= [[ 0.  0.  0.  1. -0. -0. -0.  0. -0. -1.]
 [-0.  0.  0. -0.  0. -0. -0. -0. -1.  0.]
 [ 0. -0.  1.  0. -0.  0. -0.  0.  0.  1.]
 [ 0.  0. -1.  1.  0. -0. -0. -0. -0.  0.]
 [-0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0. -0. -0. -0.  1. -0. -0. -0.  0.]
 [-0.  0.  0.  0. -0. -0.  1. -0.  0.  0.]
 [ 0.  0.  0. -0. -0. -0. -0.  1.  0.  0.]
 [-0.  0. -0.  0. -0.  0. -1.  0.  0.  0.]
 [ 1.  0.  0. -0. -0. -0. -0. -1.  0. -0.]]
R= [[ 20. -19.  17. -20.  20. -20.  16.  -8.  -4.  19.]
 [  0.   6.  -0.   1.   3.   0.   5.  -2.  10.   1.]
 [  0.   0.   4.  -5.  10. -13.  16. -17.  18.   6.]
 [  0.   0.   0.   6.  -7.   6.  -4.   2.   1.   4.]
 [  0.   0.   0.   0.   9.  -9.   5. -10.   3.  -1.]
 [  0.   0.   0.   0.   0.   9. -12.   6. -11.   2.]
 [  0.   0.   0.   0.   0.   0.  11. -14.  -2.  -9.]
 [  0.   0.   0.   0.   0.   0.   0.  15. -14.  -4.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   7. -15.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   8.]]


## Pregunta 4

Implementa el método iterativo de **Gauss-Seidel** para resolver sistemas de ecuaciones lineales de la forma:

$$
Ax = b
$$
### Parámetros

- `A`: Matriz de coeficientes $ A \in \mathbb{R}^{m \times m} $.
- `b`: Vector del lado derecho $ b \in \mathbb{R}^{m} $.
- `x0`: Vector inicial $ x^{(0)} \in \mathbb{R}^{m} $.
- `tol`: Tolerancia para el criterio de convergencia.
- `max_iter`: Número máximo de iteraciones.

### Sistema a resolver

Se debe resolver el sistema $Ax = b$, donde:

- $A \in \mathbb{R}^{1000 \times 1000}$, tal que:

$$
A[i,j] =
\begin{cases}
1001 & \text{si } i = j \\
1 & \text{si } i \ne j
\end{cases}
$$

- $b \in \mathbb{R}^{1000}$ es el vector columna:

$$
b = (1, 1, \dots, 1)^T
$$

### Retorna

- Vector $x \in \mathbb{R}^{m}$ que representa la solución aproximada al sistema $Ax = b$.




In [71]:
import numpy as np

def sust_adelante(A,b):
    """Resuelve Ax= b por sustitución hacia adelante.
    A debe ser triangular inferior """
    m=len(b)
    x=np.zeros(m)
    for i in range (m):
        aux=0
        for j in range(i):# solo hasta i-1
            aux+= A[i,j]*x[j]
        x[i]=(b[i]-aux)/A[i,i]
    return x

def gauss_seidel(A, b, xk, tol=1e-6, max_iter=100):
    m = A.shape[0]

    # Descomposición de A: A = L + D + U
    L = np.tril(A, -1)                # Parte inferior
    D = np.diag(np.diag(A))          # Diagonal principal
    U = np.triu(A, 1)                # Parte superior
    LD = L + D                       # (L + D)

    for k in range(1, max_iter + 1):
        # Fórmula teórica del método G-S hacia adelante:
        # x(k+1) = - (L + D)^(-1) · U · x(k) + (L + D)^(-1) · b

        # Primero calculamos:
        # y1 = (L + D)^(-1) · b
        # Como no podemos usar la inversa directamente,
        # resolvemos el sistema (L + D) · y1 = b
        y1 = sust_adelante(LD, b)

        # Luego calculamos:
        # y2 = -(L + D)^(-1) · (U · x(k))
        # resolviendo -(L + D) · y2 = U · x(k)
        y2 = sust_adelante(LD, U @ xk)

        # Entonces, por la fórmula original:
        # x(k+1) = -y2 + y1
        x_new = -y2 + y1

        # Cálculo del error del residuo: ||Ax - b||
        error = np.linalg.norm(A @ x_new - b)

        if error < tol:
            return x_new, k, error

        xk = x_new

    return xk, max_iter, error


# Tamaño
m = 1000

# Matriz A: 1001 en la diagonal, 1 fuera de ella
A = np.ones((m, m))
np.fill_diagonal(A, 1001)

# Vector b: todos unos
b = np.ones(m)

# Vector inicial
x0 = np.zeros(m)

# Ejecutar
raiz, iteraciones, error = gauss_seidel(A, b, x0, tol=1e-6, max_iter=100)
print("Error final:", error)


Error final: 3.578478172676417e-07
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 

## Pregunta 5

Implemente computacionalmente el método de **Jacobi** para aproximar los valores propios de una matriz simétrica $A \in \mathbb{R}^{m \times m}$.

### Fundamento teórico

El **método de Jacobi** es un algoritmo iterativo utilizado para encontrar los valores propios (autovalores) de una matriz simétrica. Este método consiste en aplicar una sucesión de rotaciones ortogonales (transformaciones de Jacobi) que eliminan los elementos fuera de la diagonal de la matriz, transformándola gradualmente en una matriz diagonal. Los elementos en la diagonal convergen a los valores propios de la matriz original.

La idea central es utilizar **rotaciones planas de Jacobi** para anular sistemáticamente los elementos fuera de la diagonal. Cada rotación afecta solo dos filas y columnas de la matriz.

El algoritmo principal realiza los siguientes pasos:

1. Calcular un ángulo $\theta$ para una posición $(i, j)$ en la matriz.  
2. Construir una matriz de rotación $G$.  
3. Aplicar la transformación $A_{k+1} = G^T A_k G$.  
4. Repetir hasta que los elementos fuera de la diagonal sean suficientemente pequeños.

La convergencia se evalúa con el criterio: $\|x_{k+1} - x_k\|_2 < \text{tol}$, donde $x_k$ es el vector de los elementos diagonales de la matriz en la iteración $k$.

---

### Sub-preguntas

**a)** Implemente la función `theta = angulo(A, i, j)` que calcule el ángulo de rotación necesario para anular el elemento $A(i, j)$:

- Si `abs(A[i, i] - A[j, j]) > 1e-16`, se utiliza:

```python
theta = 0.5 * np.arctan(2 * A[i, j] / (A[i, i] - A[j, j]))
```

- Si no, $\theta = 0$.

**Parámetros:**

- `A`: Matriz simétrica de tamaño $m \times m$  
- `i`: Índice de fila  
- `j`: Índice de columna  

**Retorna:**

- `theta`: Ángulo de rotación en radianes (`float`)

---

**b)** Implemente la función `G = matriz_rotacion(i, j, m, theta)`, que construya la matriz de rotación $G \in \mathbb{R}^{m \times m}$, con:

$G = I + Z$, donde $Z$ tiene solo las siguientes entradas no nulas:

- $Z(i,i) = \cos(\theta) - 1$  
- $Z(j,j) = \cos(\theta) - 1$  
- $Z(i,j) = -\sin(\theta)$  
- $Z(j,i) = \sin(\theta)$

**Parámetros:**

- `i`: Índice de fila  
- `j`: Índice de columna  
- `m`: Tamaño de la matriz $m \times m$  
- `theta`: Ángulo de rotación en radianes (`float`)  

**Retorna:**

- `G`: Matriz de rotación de tamaño $m \times m$ (`np.array`)

---

**c)** Implemente la función:

```python
[xk, ek] = jacobi_valores_propios(A, iterMax, tol)
```
Esta función aplica el método de Jacobi iterativamente hasta que el error entre las aproximaciones consecutivas de los valores propios sea menor que una tolerancia dada, o hasta alcanzar el número máximo de iteraciones.

**Parámetros:**

- `A`: Matriz simétrica de tamaño $m \times m$  
- `iterMax`: Número máximo de iteraciones (`int`)  
- `tol`: Tolerancia para el criterio de convergencia (`float`)  

**Retorna:**

- `xk`: Vector con los valores propios aproximados (`np.array`)  
- `ek`: Error de convergencia final $|x_{k+1} - x_k|_2$ (`float`)


In [2]:
function angle = theta(A, i, j)
    a_ij = A(i, j);
    denominator = A(i, i) - A(j, j);
    if abs(denominator) > 1e-16
        angle = 0.5 * atan(2 * a_ij / denominator);
    else
        angle = 0.0;
    end
endfunction

function G = matriz_rotacion(i, j, m, theta)
    I = eye(m);
    Z = zeros(m);
    c = cos(theta);
    s = sin(theta);
    Z(i, i) = c - 1;
    Z(j, j) = c - 1;
    if i != j
        Z(i, j) = -s;
        Z(j, i) = s;
    end
    G = I + Z;
endfunction

function valores_propios = jacobi_valores_propios(A, iterMax, tol)
    A0 = A;
    m = size(A, 1);
    Ak = A0;
    xk = diag(Ak);

    for k = 1:iterMax
        Bk = Ak;
        for i = 1:m
            for j = 1:m
                if i >= j
                    continue;
                end
                theta_ij = theta(Bk, i, j);
                G = matriz_rotacion(i, j, m, theta_ij);
                Bk = G' * Bk * G;
            end
        end
        Ak_next = Bk;
        xk_next = diag(Ak_next);
        ek = norm(xk_next - xk);
        if ek < tol
            break;
        end
        Ak = Ak_next;
        xk = xk_next;
    end
    valores_propios = diag(Ak);
endfunction

---

**d)** Para verificar el funcionamiento de la implementación del método de Jacobi, se utilizará como caso de prueba una matriz simétrica \( A \in \mathbb{R}^{15 \times 15} \), definida con la regla:

\[
A(i,j) = 0.5(i + j)
\]

Esta matriz es densa y simétrica, por lo que cumple con los requisitos del método. Se utilizará la función `jacobi_valores_propios` implementada anteriormente para calcular una aproximación de sus valores propios. Al ejecutar esta prueba, se imprimirá en consola el resultado final: el vector con las aproximaciones obtenidas y el error de convergencia asociado.

In [3]:
% --- Script principal ---
A = zeros(15);
for i = 1:15
    for j = 1:15
        A(i, j) = 0.5 * (i + j);
    end
end

valores_propios = jacobi_valores_propios(A, 1000, 1e-10);
disp("Valores propios calculados:");
disp(sort(valores_propios));

Valores propios calculados:
  -8.1909e+00
  -8.3019e-15
  -4.6512e-15
  -2.6183e-15
  -1.8486e-15
  -8.2190e-16
  -6.6880e-16
  -2.9059e-16
   7.6930e-16
   9.5559e-16
   1.6648e-15
   3.1801e-15
   3.7451e-15
   5.7077e-15
   1.2819e+02
